In [1]:
# review -> 이모티콘/자음 삭제 -> (띄워쓰기 + 맞춤법) -> 문장 분리
# - 문장 단위: 문장 부호, 종결어미, 연결조사로 사전 만들기

In [2]:
import pandas as pd
import re
import itertools

from kiwipiepy import Kiwi
from emoji import core
from hanspell import spell_checker
from pykospacing import Spacing

In [3]:
data = pd.read_csv('./data/raw/review_data.csv', encoding='utf-8-sig')

In [4]:
reviews = data['review']

In [5]:
kiwi = Kiwi()

In [6]:
# 사용자 사전 추가
kiwi.add_user_word('맛집', 'NNG', 0)
kiwi.add_user_word('라떼', 'NNG', 0)
kiwi.add_user_word('디카페인', 'NNG', 0)
# kiwi.add_user_word('카페명', 'NNP', 0) # 카페 이름 리스트가 있으면 추가

True

In [7]:
# 정규표현식 컴파일 
noun = re.compile(r'^N') # 명사형 태그
sp_char = re.compile(r'^(SS)|SW|SP|SO') # 특수문자 태그

In [8]:
# 리뷰 문장 단위로 분리
def revToSent(review):
    tmp = []
    sentences = []
    preTag = ''
    preForm = ''
    sents = kiwi.split_into_sents(review, return_tokens=True)
    for sent in sents:
        toks = sent.tokens
        for tok in toks:
            # 차후 알고리즘 수정(...)
            if bool(sp_char.match(preTag)) and bool(sp_char.match(tok.tag)):
                preTag = tok.tag
                preForm = tok.form
                continue
            
            elif bool(sp_char.match(tok.tag)):
                sentences.append(kiwi.join(tmp))
                tmp.clear()
                
                preTag = tok.tag
                preForm = tok.form
                continue
            
            condition = (preTag == 'EC') and (preForm in ['고', '지만', '은데'])
            if condition and bool(noun.match(tok.tag)):
                sentences.append(kiwi.join(tmp))
                tmp.clear()

            tmp.append(tok)
            preTag = tok.tag
            preForm = tok.form            
            
        connective = (preTag == 'EF') and (preForm in ['던데', '는데', '은데'])
        if not connective and len(tmp) > 0:
            sentences.append(kiwi.join(tmp))
            tmp.clear()
    
    return sentences

In [9]:
preprocess = []

for review in reviews:
    # print('원본 문장: ', review)
    review = core.replace_emoji(review, replace="")
    # print('특수 문자 제거: ', review)
    sents = revToSent(review)
    preprocess.append(sents)
    # print('문장 분리: ', sents)
    # print()    

In [10]:
preprocess = list(itertools.chain(*preprocess))

In [11]:
preprocess

['맛있어요',
 '만족도 최상의 커피 맛집',
 '찐 커피 맛집',
 '로마노 진짜 달달하고 맛있고',
 '아이스크림 라떼도 꼬소하고 달달하고 진짜 맛있어요!!',
 '마리토쪼도 크림 부드럽고',
 '빵도 포실포실 재방문 의사 있어요',
 '커조남 커피는 힐링입니다',
 '소금 빵도 맛있어요',
 '시나몬 롤 맛있어용',
 '하지만 소금 빵은 동굴이 없고 좀 말라 있어서 아쉬웠어요',
 '드립커피하면 커조남!',
 '오늘 드립커피는 좀 애매한 맛이긴했지만',
 '리필 커피가 맛있어서 괜춘',
 '바로 앞 공영 주차장에 주차 가능',
 '최애 커피집입니다',
 '앞산에 있을 때부터 좋아하던 곳이라 계속 가게 되네요',
 '크리스마스 페어에서도 로마노 한 잔',
 '더웠는 데 시원하게 상큼하게 잘 마셨습니다',
 '커피 맛집',
 '커피 맛을 조금 아는 남자가 반갑게 맞아 주는 찐 커피 맛집!!',
 '리모델링하고는 처음 가 봤는데 인테리어도 더 이쁘고',
 '디저트도 많아진 거 같아서 좋아요!!',
 '커피는 여전히 너무 맛있네요',
 '로마노는 처음 마셔 봤는데 완전 신세계!!!',
 '레몬 맛 크림에 에스프레소라니 완전 취저 당했어요',
 '디카페인 라떼도 생각보다 맛있어서 충격적..',
 '디카페인 맛있는 카페 없던데 여기는 찐이네요',
 '커피도 맛나고',
 '리필도 되니 늠 좋네요',
 '주말마다 오는 커피집',
 '커피가 너무 맛있어요.',
 '디카페인 커피 원두 시음해 보고 맛있어서 구매했습니다',
 '',
 '드립커피는 별로 안 좋아해서..',
 '친구는 괜찮대요.',
 '갈 만은해요',
 '두 번째 방문입니다.',
 '일부러 들렀어요.',
 '전에 마셔 보지 못한 카페오레가 궁금해서요.',
 '카페오레 온도가 참 좋아요.',
 '부드러움도요.',
 '소금 빵과 잘 어울리는 맛입니다.',
 '커조남모르 시는 대구부인들 계신가용',
 '?',
 '동성로 점도 넘 좋았지만',
 '본점은 정말 최고인 것 같아요',
 '평일에도 항상 사람들 많고',
 

In [12]:
spacing = Spacing()
final = []

for sent_i in preprocess:
    new_sent_i = spell_checker.check(sent_i).checked
    new_sent_i = spacing(new_sent_i)
    # print('원본 : ', sent_i)
    # print('맞춤법+띄어 : ', new_sent_i)
    # print()
    final.append(new_sent_i)

1/1 [==============================] - 0s 37ms/step


In [13]:
df = pd.DataFrame({'reviews': preprocess})
df.to_csv('./data/train_x.csv', index=False, encoding='utf-8-sig')

In [14]:
len(preprocess)

2177

In [15]:
df = pd.DataFrame({'reviews': final})
df.to_csv('./data/train_x_spellcheck.csv', index=False, encoding='utf-8-sig')